In [1]:
import ctypes

class DynamicArray:
    """
    A dynamic array implementation in Python.
    This class mimics the behavior of Python's built-in list,
    demonstrating how the underlying array resizes itself.
    """

    def __init__(self):
        """
        Initializes an empty dynamic array.
        _n: current number of elements in the array
        _capacity: current total capacity of the array
        _A: the underlying fixed-size array (using ctypes for low-level array)
        """
        self._n = 0  # Number of elements
        self._capacity = 1  # Initial capacity
        self._A = self._make_array(self._capacity) # Underlying array

    def __len__(self):
        """
        Returns the number of elements in the array.
        """
        return self._n

    def __getitem__(self, k):
        """
        Returns the element at index k.
        Raises IndexError if k is out of bounds.
        """
        if not 0 <= k < self._n:
            raise IndexError('invalid index')
        return self._A[k]

    def __setitem__(self, k, value):
        """
        Sets the element at index k to value.
        Raises IndexError if k is out of bounds.
        """
        if not 0 <= k < self._n:
            raise IndexError('invalid index')
        self._A[k] = value

    def append(self, obj):
        """
        Adds an object to the end of the array.
        Resizes if the current capacity is full.
        """
        if self._n == self._capacity:
            # Time to resize: double the capacity
            self._resize(2 * self._capacity)
        self._A[self._n] = obj
        self._n += 1

    def extend(self, iterable):
        """
        Extend the array by appending all the items from the iterable.
        """
        for item in iterable:
            self.append(item)

    def _resize(self, new_capacity):
        """
        Resizes the underlying array to a new capacity.
        Creates a new array of the specified capacity,
        copies all existing elements to the new array,
        and then updates the array reference.
        """
        B = self._make_array(new_capacity) # New array
        for k in range(self._n):
            B[k] = self._A[k] # Copy existing elements
        self._A = B # Update reference to new array
        self._capacity = new_capacity # Update capacity

    def _make_array(self, capacity):
        """
        Returns a new low-level array (using ctypes) with the given capacity.
        This simulates a contiguous block of memory.
        """
        return (capacity * ctypes.py_object)()

    def insert(self, k, value):
        """
        Inserts a value at a specific index k, shifting subsequent elements.
        Raises IndexError if k is out of bounds.
        """
        if not 0 <= k <= self._n:
            raise IndexError('invalid index')
        if self._n == self._capacity:
            self._resize(2 * self._capacity) # Double capacity if full
        for j in range(self._n, k, -1): # Shift elements from right to left
            self._A[j] = self._A[j-1]
        self._A[k] = value # Place new item
        self._n += 1

    def remove(self, value):
        """
        Removes the first occurrence of a value from the array.
        Raises ValueError if the value is not found.
        """
        found = False
        for k in range(self._n):
            if self._A[k] == value: # Found a match
                for j in range(k, self._n - 1): # Shift elements from left to right
                    self._A[j] = self._A[j+1]
                self._A[self._n - 1] = None # Help garbage collection
                self._n -= 1
                found = True
                break # Exit after removing first occurrence
        if not found:
            raise ValueError('value not found')

    def pop(self, k=None):
        """
        Removes and returns the element at index k.
        If k is not specified, removes and returns the last element.
        Raises IndexError if k is out of bounds or array is empty.
        """
        if self._n == 0:
            raise IndexError('pop from empty array')
        if k is None:
            k = self._n - 1 # Default to last element
        # Handle negative indices
        if k < 0:
            k = self._n + k
        if not 0 <= k < self._n:
            raise IndexError('invalid index')

        value = self._A[k]
        for j in range(k, self._n - 1): # Shift elements
            self._A[j] = self._A[j+1]
        self._A[self._n - 1] = None # Help garbage collection
        self._n -= 1

        # Optional: Shrink array if it becomes too sparse
        # This prevents excessive memory usage after many pops
        if self._n > 0 and self._n < self._capacity // 4:
            self._resize(self._capacity // 2)
        elif self._n == 0 and self._capacity > 1: # If empty, reset capacity to 1
            self._resize(1)

        return value

    def clear(self):
        """
        Removes all elements from the array.
        Resets the array to its initial empty state.
        """
        self._n = 0
        self._capacity = 1
        self._A = self._make_array(self._capacity)

    def index(self, value, start=0, end=None):
        """
        Returns the index of the first occurrence of value.
        Raises ValueError if the value is not present.
        Optional arguments start and end are interpreted as in slice notation.
        """
        if start < 0:
            start = 0
        if end is None or end > self._n:
            end = self._n

        for k in range(start, end):
            if self._A[k] == value:
                return k
        raise ValueError(f'{value} is not in array')

    def count(self, value):
        """
        Returns the number of times value appears in the array.
        """
        count = 0
        for k in range(self._n):
            if self._A[k] == value:
                count += 1
        return count

    def reverse(self):
        """
        Reverses the elements of the array in place.
        """
        left, right = 0, self._n - 1
        while left < right:
            self._A[left], self._A[right] = self._A[right], self._A[left]
            left += 1
            right -= 1

    def copy(self):
        """
        Returns a shallow copy of the array.
        """
        new_array = DynamicArray()
        new_array._n = self._n
        new_array._capacity = self._capacity
        new_array._A = self._make_array(self._capacity)
        for k in range(self._n):
            new_array._A[k] = self._A[k]
        return new_array

    def sort(self, key=None, reverse=False):
        """
        Sorts the elements of the array in place.
        Uses Python's built-in Timsort algorithm for efficiency.
        Optional arguments key and reverse are interpreted as in list.sort().
        """
        # Create a temporary list of actual elements for sorting
        temp_list = [self._A[i] for i in range(self._n)]

        # Use Python's built-in sort for simplicity and efficiency
        temp_list.sort(key=key, reverse=reverse)

        # Copy sorted elements back to the underlying array
        for i in range(self._n):
            self._A[i] = temp_list[i]


    def __str__(self):
        """
        Returns a string representation of the dynamic array.
        """
        elements = [str(self._A[i]) for i in range(self._n)]
        return '[' + ', '.join(elements) + ']'

    def __repr__(self):
        """
        Returns a developer-friendly string representation.
        """
        elements_repr = ', '.join(repr(self._A[i]) for i in range(self._n))
        return f"DynamicArray([{elements_repr}], capacity={self._capacity})"


# --- Example Usage ---
if __name__ == "__main__":
    print("Initializing DynamicArray...")
    arr = DynamicArray()
    print(f"Initial array: {arr}, Length: {len(arr)}, Capacity: {arr._capacity}")

    print("\n--- Testing append and extend ---")
    arr.append(10)
    arr.append(20)
    arr.extend([30, 40, 50])
    print(f"After append(10,20) and extend([30,40,50]): {arr}, Length: {len(arr)}, Capacity: {arr._capacity}")

    print(f"\nElement at index 2: {arr[2]}") # Should be 30
    arr[2] = 35 # Test __setitem__
    print(f"After arr[2] = 35: {arr}")

    print("\n--- Testing insert ---")
    arr.insert(1, 15) # Insert 15 at index 1
    print(f"After insert(1, 15): {arr}, Length: {len(arr)}, Capacity: {arr._capacity}")
    arr.insert(0, 5) # Insert at beginning
    print(f"After insert(0, 5): {arr}, Length: {len(arr)}, Capacity: {arr._capacity}")
    arr.insert(len(arr), 60) # Insert at end
    print(f"After insert(len(arr), 60): {arr}, Length: {len(arr)}, Capacity: {arr._capacity}")


    print("\n--- Testing remove and pop ---")
    arr.remove(35) # Remove 35
    print(f"After remove(35): {arr}, Length: {len(arr)}, Capacity: {arr._capacity}")

    popped_item = arr.pop() # Pop last
    print(f"Popped item (last): {popped_item}")
    print(f"Array after pop(): {arr}, Length: {len(arr)}, Capacity: {arr._capacity}")

    popped_item = arr.pop(0) # Pop first
    print(f"Popped item (index 0): {popped_item}")
    print(f"Array after pop(0): {arr}, Length: {len(arr)}, Capacity: {arr._capacity}")

    popped_item = arr.pop(-2) # Pop second to last using negative index
    print(f"Popped item (index -2): {popped_item}")
    print(f"Array after pop(-2): {arr}, Length: {len(arr)}, Capacity: {arr._capacity}")


    print("\n--- Testing count and index ---")
    arr.extend([10, 20, 10, 70])
    print(f"Array for count/index test: {arr}")
    print(f"Count of 10: {arr.count(10)}") # Should be 2
    print(f"Index of 20: {arr.index(20)}") # Should be 1 (first 20)
    try:
        print(f"Index of 10 from index 2: {arr.index(10, 2)}") # Should be 4
    except ValueError as e:
        print(e)
    try:
        arr.index(999)
    except ValueError as e:
        print(f"Attempted to find index of non-existent value: {e}")


    print("\n--- Testing reverse ---")
    print(f"Original array: {arr}")
    arr.reverse()
    print(f"Reversed array: {arr}")

    print("\n--- Testing copy ---")
    arr_copy = arr.copy()
    print(f"Original array: {arr}")
    print(f"Copied array: {arr_copy}")
    arr_copy.append(100)
    print(f"Copied array after append(100): {arr_copy}")
    print(f"Original array (should be unchanged): {arr}") # Should be unchanged


    print("\n--- Testing sort ---")
    unsorted_arr = DynamicArray()
    unsorted_arr.extend([50, 10, 40, 20, 30])
    print(f"Unsorted array: {unsorted_arr}")
    unsorted_arr.sort()
    print(f"Sorted array (ascending): {unsorted_arr}")

    unsorted_arr.extend([1, 9, 3, 7, 5])
    print(f"Unsorted array for reverse sort: {unsorted_arr}")
    unsorted_arr.sort(reverse=True)
    print(f"Sorted array (descending): {unsorted_arr}")

    # Test with key
    class Person:
        def __init__(self, name, age):
            self.name = name
            self.age = age
        def __repr__(self):
            return f"Person({self.name}, {self.age})"

    people_arr = DynamicArray()
    people_arr.append(Person("Alice", 30))
    people_arr.append(Person("Bob", 25))
    people_arr.append(Person("Charlie", 35))
    print(f"People array (unsorted): {people_arr}")
    people_arr.sort(key=lambda p: p.age)
    print(f"People array (sorted by age): {people_arr}")


    print("\n--- Testing clear ---")
    print(f"Array before clear: {arr}")
    arr.clear()
    print(f"Array after clear: {arr}, Length: {len(arr)}, Capacity: {arr._capacity}")

    print("\n--- Testing edge cases for pop (shrinking)...")
    arr.append(1)
    arr.append(2)
    arr.append(3)
    arr.append(4)
    print(f"Array: {arr}, Length: {len(arr)}, Capacity: {arr._capacity}")
    arr.pop()
    print(f"After pop: {arr}, Length: {len(arr)}, Capacity: {arr._capacity}") # Capacity should remain 4
    arr.pop()
    print(f"After pop: {arr}, Length: {len(arr)}, Capacity: {arr._capacity}") # Capacity should shrink to 2
    arr.pop()
    print(f"After pop: {arr}, Length: {len(arr)}, Capacity: {arr._capacity}") # Capacity should shrink to 1

    try:
        arr.pop()
    except IndexError as e:
        print(f"\nAttempted to pop from empty array: {e}")

    try:
        arr[5]
    except IndexError as e:
        print(f"Attempted to access invalid index: {e}")

    try:
        arr.remove(999)
    except ValueError as e:
        print(f"Attempted to remove non-existent value: {e}")

Initializing DynamicArray...
Initial array: [], Length: 0, Capacity: 1

--- Testing append and extend ---
After append(10,20) and extend([30,40,50]): [10, 20, 30, 40, 50], Length: 5, Capacity: 8

Element at index 2: 30
After arr[2] = 35: [10, 20, 35, 40, 50]

--- Testing insert ---
After insert(1, 15): [10, 15, 20, 35, 40, 50], Length: 6, Capacity: 8
After insert(0, 5): [5, 10, 15, 20, 35, 40, 50], Length: 7, Capacity: 8
After insert(len(arr), 60): [5, 10, 15, 20, 35, 40, 50, 60], Length: 8, Capacity: 8

--- Testing remove and pop ---
After remove(35): [5, 10, 15, 20, 40, 50, 60], Length: 7, Capacity: 8
Popped item (last): 60
Array after pop(): [5, 10, 15, 20, 40, 50], Length: 6, Capacity: 8
Popped item (index 0): 5
Array after pop(0): [10, 15, 20, 40, 50], Length: 5, Capacity: 8
Popped item (index -2): 40
Array after pop(-2): [10, 15, 20, 50], Length: 4, Capacity: 8

--- Testing count and index ---
Array for count/index test: [10, 15, 20, 50, 10, 20, 10, 70]
Count of 10: 3
Index of 20